In [ ]:
def create_evaluation_table(df):
    rows_to_add = []  # List to store each valid row

    for idx, row in df.iterrows():
        data_tag_flag = False
        my_tag_flag = False
        
        try:
            if row['Tags'][0] != '':
                data_tag_flag = True
        except:
            pass
        try:
            if row['My_Sentiment_Tags'][0] != '':
                my_tag_flag = True
        except:
            pass

        if data_tag_flag and my_tag_flag:
            rows_to_add.append(row)  # Add the current row to the list

    if rows_to_add:
        eval_df = pd.concat([pd.DataFrame([row]) for row in rows_to_add], ignore_index=True)
        eval_df = eval_df[['Tags', 'My_Sentiment_Tags']]
    else:
        eval_df = pd.DataFrame(columns=['Tags', 'My_Sentiment_Tags'])

    return eval_df



def extract_tags(sentiment_string):
    """Extracts tags and sentiments from a string using regex."""
    if isinstance(sentiment_string, list):
        sentiment_string = ', '.join(sentiment_string)
    return {match[0]: int(match[1]) for match in re.findall(r'(\w+)\[([+-]?\d+)\]', sentiment_string)}



def process_tags(df, key_mapping):
    """Processes tags by applying a key mapping and returns a DataFrame with relevant tag information."""
    results = []
    reverse_mapping = {v: k for k, v in key_mapping.items()}

    for idx, row in df.iterrows():
        tags = extract_tags(row['Tags'])
        my_sentiment_tags = extract_tags(row['My_Sentiment_Tags'])
        mapped_features = {reverse_mapping.get(feature, feature): sentiments for feature, sentiments in my_sentiment_tags.items()}

        for tag, sentiment in tags.items():
            mapped_tag = reverse_mapping.get(tag, tag)
            if mapped_tag in mapped_features:
                results.append({
                    'Index': idx,
                    'Feature': mapped_tag,
                    'Tags Sentiment': sentiment,
                    'My Sentiment Tags Sentiment': mapped_features[mapped_tag]
                })

    return pd.DataFrame(results)



def compute_confusion_matrix(result_df):
    """Computes the confusion matrix from the processed tag data."""
    TP = FP = TN = FN = 0

    for index, row in result_df.iterrows():
        tag_sent = row['Tags Sentiment']
        my_sent = row['My Sentiment Tags Sentiment']
        
        if tag_sent > 0 and my_sent > 0:
            TP += 1
        elif tag_sent <= 0 and my_sent > 0:
            FP += 1
        elif tag_sent <= 0 and my_sent <= 0:
            TN += 1
        elif tag_sent > 0 and my_sent <= 0:
            FN += 1

    return pd.DataFrame({
        'TP': [TP],
        'FP': [FP],
        'TN': [TN],
        'FN': [FN]
    }, index=['Overall'])



def calculate_metrics(conf_matrix):
    TP = conf_matrix.loc['Overall', 'TP']
    FP = conf_matrix.loc['Overall', 'FP']
    TN = conf_matrix.loc['Overall', 'TN']
    FN = conf_matrix.loc['Overall', 'FN']
    
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    recall = TP / (TP + FN) if TP + FN != 0 else 0
    accuracy = (TP + TN) / (TP + FP + TN + FN) if TP + FP + TN + FN != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0
    
    return pd.DataFrame({
        'Precision': [round(precision, 3)],
        'Recall': [round(recall, 3)],
        'Accuracy': [round(accuracy, 3)],
        'F1 Score': [round(f1_score, 3)]
    }, index=['Overall'])



evaluation_table = create_evaluation_table(df)
result_df = process_tags(evaluation_table, key_mapping)
conf_matrix_df = compute_confusion_matrix(result_df)
metrics_df = calculate_metrics(conf_matrix_df)
print(conf_matrix_df)
print(metrics_df)